In [ ]:
import boto3
import json
client = boto3.client('sqs')

In [ ]:
response = client.list_queues()
queues = response['QueueUrls']

In [ ]:
for queue in queues: 
    client.receive_message(queue, 
                           AttributeNames = ['All'],
                           MessageAttributeNames = [''])

In [45]:
sqs = boto3.resource('sqs')
queue_name = 'TextractPipelineStack-AsyncJobsE9347181-32TIZOY3ZZ9B'
queue = sqs.get_queue_by_name(QueueName=queue_name)
queue

sqs.Queue(url='https://eu-west-1.queue.amazonaws.com/228802818628/TextractPipelineStack-AsyncJobsE9347181-32TIZOY3ZZ9B')

In [44]:
max_queue_messages = 10
message_bodies = []
queue = sqs.get_queue_by_name(QueueName=queue_name)
while True:
    messages_to_delete = []
    for message in queue.receive_messages(
            MaxNumberOfMessages=max_queue_messages):
        # process message body
        body = json.loads(message.body)
        message_bodies.append(body)

KeyboardInterrupt: 

In [32]:
documentId_list = []
for message in message_bodies:
    documentId_list.append(message['documentId'])


In [37]:
def get_messages_from_queue(queue_url):
    sqs_client = boto3.client('sqs')

    messages = []

    while True:
        resp = sqs_client.receive_message(
            QueueUrl=queue_url,
            AttributeNames=['All'],
            MaxNumberOfMessages=10
        )

        try:
            messages.extend(resp['Messages'])
        except KeyError:
            break

        entries = [
            {'Id': msg['MessageId'], 'ReceiptHandle': msg['ReceiptHandle']}
            for msg in resp['Messages']
        ]

        resp = sqs_client.delete_message_batch(
            QueueUrl=queue_url, Entries=entries
        )

        if len(resp['Successful']) != len(entries):
            raise RuntimeError(
                f"Failed to delete messages: entries={entries!r} resp={resp!r}"
            )

    return messages


In [43]:
queueUrl = 'https://eu-west-1.queue.amazonaws.com/228802818628/TextractPipelineStack-AsyncJobsE9347181-32TIZOY3ZZ9B'
messages = get_messages_from_queue(queueUrl)

In [40]:
len(messages)

295

In [42]:
messages[0]

{'MessageId': '14213796-39dd-46b8-9a8f-6969ca8f1e66',
 'ReceiptHandle': 'AQEBZvEDSSvjmjidtpfkI1nkkjT24W5KmKvDE1a4k5yv2dR2d1e0XQfVQBAdFPGu+L4A5rIina64HrLcv+lJs9TV5wIxpPGHeragEpiM0WN/Q0R3LAVG86DmzW4RFXN2wDVNE9q9ntkRkrK/ZMNmjIsPvMiMTywdELE9cLPMCQxQJMisPRCd3ZIIkeaQH4hCHELAkmpmTLIZWVldlqXiEs23eoUz1F2c2AK5whP+t2OHzWYcZ7lK3zQmWzjYQf3PFExxdcrQZYMPv2fp83jYL0vJgQgGZN+oOf6wKgVnPuv62M5wXuA7iR3xDESaXowFjwRzRfvoVrxJ5bOyvKUsVgJpG15mzjtCxBMLS3ctm6U9ALciQW4zBMov6LVLK0OFOny9ybz78cMnj2Cxs3o39oNbMQEbgtEl4gtSS5016rDGORUzjWrrgLQHD5G5A55ZJTby',
 'MD5OfBody': '361519fb7e3c59639dba4ee008685ea6',
 'Body': '{"documentId": "f7271510-7f0f-11ea-8f53-ae6630b05962", "features": ["Text", "Forms", "Tables"], "bucketName": "textractpipelinestack-documentsbucket9ec9deb9-1adc6crt86qxu", "objectName": "public/Jeroen_Taxology_3e8ae524-d722-40da-b0ab-767d36b12a1e.pdf"}',
 'Attributes': {'SenderId': 'AROATKRNV6JCFIG7GHW6B:TextractPipelineStack-TaskProcessor26BEA722-1XKYYTW7EUVRB',
  'ApproximateFirstReceiveTimestamp': '158702